In [1]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

from pyomo.core.util import quicksum

import pandas as pd
from datetime import date, datetime
import itertools

from bank_schedule.data import Data

In [2]:
data = Data('/Users/sykuznetsov/Documents/GitHub/bank_schedule/data/raw')

In [15]:
tnp

,TID,money
0,406136,160000
1,406139,387000
2,406145,287000
3,406148,355000
4,406180,597000
...,...,...
1625,699578,47000
1626,699579,236000
1627,699629,67000
1628,699641,278000


In [82]:
money_in = data.get_money_in()

money_in = money_in[(money_in["date"].dt.date >= date(2022, 11, 1)) & (money_in["date"].dt.date <= date(2022, 11, 3))]

date_num_dict = {date: num for num, date in enumerate(sorted(money_in['date'].unique()))}

money_in['date'] = money_in['date'].map(date_num_dict)

params = data.get_params_dict()

TIDS = money_in['TID'].unique()
DATES = money_in['date'].unique()

quantity_cars = 6

kwargs = {}

M = 100000000000

days_from_inc_dict = {tid: 0 for tid in TIDS}

money_in_dict = money_in.set_index(['TID', 'date']).to_dict()['money_in']


DATES

/var/folders/53/2dkgx2ms60940cz9185gp4xc0000gq/T/ipykernel_41421/1900490213.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  money_in['date'] = money_in['date'].map(date_num_dict)


array([0, 1, 2])

In [89]:
params['day_end'] - params['day_start']

TypeError: unsupported operand type(s) for -: 'datetime.time' and 'datetime.time'

In [92]:
params['day_end'].

datetime.time(20, 0)

In [86]:

# BIG_NUM = 40*60
# postomat_places = list(fixed_points) + list(possible_postomats)
# center_mass_set = set(distances['id_center_mass'].to_list())
# metro_points_set = set(distanses_metro['object_id_metro'].to_list())

# distances_dict = {(id_center_mass, postomat_place_id): walk_time for _, postomat_place_id, id_center_mass , _, _, walk_time in distances.itertuples()}
# distances_metro_dict = {(object_id_metro, object_id): walk_time for _, object_id, object_id_metro ,  _, walk_time in distanses_metro.itertuples()}



# Создание конкретной модели pyomo
model = pyo.ConcreteModel()

model.MAX_MONEY = params['max_money']

model.MAX_DAYS_INC = params['max_days_inc']

model.OVERNIGHT_BY_DAY = params['overnight']/100/365

model.COST_INC_PERS = params['cost_inc_pers']

model.COST_INC_MIN = params['cost_inc_min']

model.days_from_inc_dict = days_from_inc_dict

model.money_in_dict = money_in_dict

model.money_start_dict = {row[1]['TID']: row[1]['money'] for row in data.get_money_start().iterrows()}


# Переменные
model.money_inc = pyo.Var(TIDS, DATES, within=pyo.Binary, initialize=0)


model.money_inside_TID = pyo.Var(TIDS, DATES,  within=pyo.NonNegativeReals, initialize=0 )

def con_money_inside_TID_1(model, tid, date):
    if date == 0:
        # TODO Проверить не теряем ли мы день
        out = model.money_start_dict[tid]
    else:
        out = model.money_inside_TID[tid, date - 1] + model.money_in_dict[(tid, date - 1)]
    return  out -  model.money_inc[tid, date] * M <= model.money_inside_TID[tid, date]
model.con_money_inside_TID_1 = pyo.Constraint(TIDS, DATES, rule=con_money_inside_TID_1)



def con_money_inside_TID_2(model, tid, date):
    return model.money_inside_TID[tid, date] <= (1 - model.money_inc[tid, date]) * M
model.con_money_inside_TID_2 = pyo.Constraint(TIDS, DATES, rule=con_money_inside_TID_2)


def con_max_money(model, tid, date):
    return model.money_inside_TID[tid, date] <= model.MAX_MONEY
model.con_max_money = pyo.Constraint(TIDS, DATES, rule=con_max_money)




model.days_from_inc = pyo.Var(TIDS, DATES,  within=pyo.NonNegativeIntegers, initialize=0 )

# Можно не водить переменные, а просто ограничиться ограничением
def con_days_from_inc(model, tid, date):

    if date == 0:
        out = days_from_inc_dict[tid]
    else:
        out = model.days_from_inc[tid, date - 1] + 1


    return out -  model.money_inc[tid, date] * M <= model.days_from_inc[tid, date]



model.con_days_from_inc = pyo.Constraint(TIDS, DATES, rule=con_days_from_inc)


def con_max_days_inc(model, tid, date):
    return model.days_from_inc[tid, date] <= model.MAX_DAYS_INC
model.con_max_days_inc = pyo.Constraint(TIDS, DATES, rule=con_max_days_inc)



def costs_from_money(model, tid, date):
    return model.money_inside_TID[tid, date] * model.OVERNIGHT_BY_DAY
model.costs_from_money = pyo.Expression(TIDS, DATES, rule=costs_from_money )



model.costs_for_inc = pyo.Var(TIDS, DATES,  within=pyo.NonNegativeReals, initialize=0 )
def con_costs_from_money_1(model, tid, date):
    return model.money_inc[tid, date] * model.COST_INC_MIN <= model.costs_for_inc[tid, date]
model.con_costs_from_money_1 = pyo.Constraint(TIDS, DATES, rule=con_costs_from_money_1 )

def con_costs_from_money_2(model, tid, date):
    return model.money_inside_TID[tid, date] * model.COST_INC_PERS -  model.money_inc[tid, date] * M <= model.costs_for_inc[tid, date]
model.con_costs_from_money_2 = pyo.Constraint(TIDS, DATES, rule=con_costs_from_money_2 )







model.OBJ = pyo.Objective(expr=quicksum([ model.money_inside_TID[tid, date] for tid, date in itertools.product(TIDS, DATES) ]) +
                                quicksum([model.days_from_inc[tid, date] for tid, date in itertools.product(TIDS, DATES) ]) +
                                quicksum([model.costs_from_money[tid, date] for tid, date in itertools.product(TIDS, DATES) ]) +
                                quicksum([model.costs_for_inc[tid, date] for tid, date in itertools.product(TIDS, DATES) ]), sense=pyo.minimize)
# minimize

# , executable="/usr/local/Cellar/cbc/2.10.8/bin/cbc"
opt = SolverFactory('cbc')

for key in kwargs:
    opt.options[key] = kwargs[key]

results = opt.solve(model)


print(results['Solver'])
print(model.money_inside_TID.extract_values())
print(model.costs_from_money.extract_values())
print(model.days_from_inc.extract_values())

print(model.costs_for_inc.extract_values())



solution_dict = model.money_inc.extract_values()
solution_pd = pd.DataFrame(solution_dict.items(), columns=['index', 'inc'])
solution_pd['TID'], solution_pd['date_num'] = zip(*solution_pd['index'])

display(solution_pd)


- Status: ok
  User time: -1.0
  System time: 21.33
  Wallclock time: 23.8
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 23
      Number of created subproblems: 23
    Black box: 
      Number of iterations: 6659
  Error rc: 0
  Time: 23.831594944000244

{(638981, 0): 0.0, (638981, 1): 0.0, (638981, 2): 0.0, (688211, 0): 0.0, (688211, 1): 0.0, (688211, 2): 0.0, (688213, 0): 0.0, (688213, 1): 0.0, (688213, 2): 0.0, (688214, 0): 0.0, (688214, 1): 0.0, (688214, 2): 0.0, (688219, 0): 0.0, (688219, 1): 0.0, (688219, 2): 0.0, (688223, 0): 0.0, (688223, 1): 0.0, (688223, 2): 0.0, (688225, 0): 0.0, (688225, 1): 0.0, (688225, 2): 0.0, (688226, 0): 0.0, (688226, 1): 0.0, (688226, 2): 0.0, (688241, 0): 0.0, (688241, 1): 0.0, (688241, 2): 0.0, (688242, 0): 0.0, (688242, 1): 0.0, (688242, 2): 0.0, (688247, 0): 0.0, (68824

,index,inc,TID,date_num
0,"(638981, 0)",1.0,638981,0
1,"(638981, 1)",1.0,638981,1
2,"(638981, 2)",0.0,638981,2
3,"(688211, 0)",1.0,688211,0
4,"(688211, 1)",1.0,688211,1
...,...,...,...,...
4885,"(606198, 1)",1.0,606198,1
4886,"(606198, 2)",1.0,606198,2
4887,"(606199, 0)",1.0,606199,0
4888,"(606199, 1)",1.0,606199,1


In [94]:
print(model.costs_from_money.extract_values())

{(638981, 0): <pyomo.core.expr.numeric_expr.MonomialTermExpression object at 0x12d843ca0>, (638981, 1): <pyomo.core.expr.numeric_expr.MonomialTermExpression object at 0x12d843d30>, (638981, 2): <pyomo.core.expr.numeric_expr.MonomialTermExpression object at 0x12d843a90>, (688211, 0): <pyomo.core.expr.numeric_expr.MonomialTermExpression object at 0x12d843a60>, (688211, 1): <pyomo.core.expr.numeric_expr.MonomialTermExpression object at 0x12d843a30>, (688211, 2): <pyomo.core.expr.numeric_expr.MonomialTermExpression object at 0x12d843a00>, (688213, 0): <pyomo.core.expr.numeric_expr.MonomialTermExpression object at 0x12d8439d0>, (688213, 1): <pyomo.core.expr.numeric_expr.MonomialTermExpression object at 0x12d8439a0>, (688213, 2): <pyomo.core.expr.numeric_expr.MonomialTermExpression object at 0x12d843970>, (688214, 0): <pyomo.core.expr.numeric_expr.MonomialTermExpression object at 0x12d843940>, (688214, 1): <pyomo.core.expr.numeric_expr.MonomialTermExpression object at 0x12d843910>, (688214, 

{(638981, 0): 0.0, (638981, 1): 0.0, (638981, 2): 0.0, (638981, 3): 0.0, (638981, 4): 0.0, (638981, 5): 0.0, (638981, 6): 0.0, (688211, 0): 0.0, (688211, 1): 0.0, (688211, 2): 0.0, (688211, 3): 0.0, (688211, 4): 0.0, (688211, 5): 0.0, (688211, 6): 0.0, (688213, 0): 0.0, (688213, 1): 0.0, (688213, 2): 0.0, (688213, 3): 0.0, (688213, 4): 0.0, (688213, 5): 0.0, (688213, 6): 0.0, (688214, 0): 0.0, (688214, 1): 0.0, (688214, 2): 0.0, (688214, 3): 0.0, (688214, 4): 0.0, (688214, 5): 0.0, (688214, 6): 0.0, (688219, 0): 0.0, (688219, 1): 0.0, (688219, 2): 0.0, (688219, 3): 0.0, (688219, 4): 0.0, (688219, 5): 0.0, (688219, 6): 0.0, (688223, 0): 0.0, (688223, 1): 0.0, (688223, 2): 0.0, (688223, 3): 0.0, (688223, 4): 0.0, (688223, 5): 0.0, (688223, 6): 0.0, (688225, 0): 0.0, (688225, 1): 0.0, (688225, 2): 0.0, (688225, 3): 0.0, (688225, 4): 0.0, (688225, 5): 0.0, (688225, 6): 0.0, (688226, 0): 0.0, (688226, 1): 0.0, (688226, 2): 0.0, (688226, 3): 0.0, (688226, 4): 0.0, (688226, 5): 0.0, (688226, 

,index,inc,TID,date_num
0,"(638981, 0)",1.0,638981,0
1,"(638981, 1)",1.0,638981,1
2,"(638981, 2)",1.0,638981,2
3,"(638981, 3)",1.0,638981,3
4,"(638981, 4)",1.0,638981,4
...,...,...,...,...
11405,"(606199, 2)",1.0,606199,2
11406,"(606199, 3)",1.0,606199,3
11407,"(606199, 4)",1.0,606199,4
11408,"(606199, 5)",1.0,606199,5


,index,inc,TID,date_num
0,"(638981, 0)",0.0,638981,0
1,"(638981, 1)",0.0,638981,1
2,"(638981, 2)",0.0,638981,2
3,"(638981, 3)",0.0,638981,3
4,"(638981, 4)",0.0,638981,4
...,...,...,...,...
11405,"(606199, 2)",0.0,606199,2
11406,"(606199, 3)",0.0,606199,3
11407,"(606199, 4)",0.0,606199,4
11408,"(606199, 5)",0.0,606199,5


In [43]:
solution_pd['inc'].sum()

0.0

In [34]:
solution_dict

{(638981, 0): 0.0,
 (638981, 1): 0.0,
 (638981, 2): 0.0,
 (638981, 3): 0.0,
 (638981, 4): 0.0,
 (638981, 5): 0.0,
 (638981, 6): 0.0,
 (688211, 0): 0.0,
 (688211, 1): 0.0,
 (688211, 2): 0.0,
 (688211, 3): 0.0,
 (688211, 4): 0.0,
 (688211, 5): 0.0,
 (688211, 6): 0.0,
 (688213, 0): 0.0,
 (688213, 1): 0.0,
 (688213, 2): 0.0,
 (688213, 3): 0.0,
 (688213, 4): 0.0,
 (688213, 5): 0.0,
 (688213, 6): 0.0,
 (688214, 0): 0.0,
 (688214, 1): 0.0,
 (688214, 2): 0.0,
 (688214, 3): 0.0,
 (688214, 4): 0.0,
 (688214, 5): 0.0,
 (688214, 6): 0.0,
 (688219, 0): 0.0,
 (688219, 1): 0.0,
 (688219, 2): 0.0,
 (688219, 3): 0.0,
 (688219, 4): 0.0,
 (688219, 5): 0.0,
 (688219, 6): 0.0,
 (688223, 0): 0.0,
 (688223, 1): 0.0,
 (688223, 2): 0.0,
 (688223, 3): 0.0,
 (688223, 4): 0.0,
 (688223, 5): 0.0,
 (688223, 6): 0.0,
 (688225, 0): 0.0,
 (688225, 1): 0.0,
 (688225, 2): 0.0,
 (688225, 3): 0.0,
 (688225, 4): 0.0,
 (688225, 5): 0.0,
 (688225, 6): 0.0,
 (688226, 0): 0.0,
 (688226, 1): 0.0,
 (688226, 2): 0.0,
 (688226, 3)

In [ ]:

    return solution_pd.loc[solution_pd['place_postomat'] > 0, 'object_id'].to_list()

# BIG_NUM = 40*60
# postomat_places = list(fixed_points) + list(possible_postomats)
# center_mass_set = set(distances['id_center_mass'].to_list())
# metro_points_set = set(distanses_metro['object_id_metro'].to_list())

# distances_dict = {(id_center_mass, postomat_place_id): walk_time for _, postomat_place_id, id_center_mass , _, _, walk_time in distances.itertuples()}
# distances_metro_dict = {(object_id_metro, object_id): walk_time for _, object_id, object_id_metro ,  _, walk_time in distanses_metro.itertuples()}

# Создание конкретной модели pyomo
model = pyo.ConcreteModel()

# Переменные
model.has_postomat = pyo.Var(postomat_places, within=pyo.Binary, initialize=0)

if precalculated_points is not None:
    for point in precalculated_points:
        model.has_postomat[point] = 1


for fixed_point in fixed_points:
    model.has_postomat[fixed_point].fix(1)

model.center_mass_time_to_nearest_postamat = pyo.Var(population_points, within=pyo.NonNegativeReals)

#Ограничения

def con_center_mass_time_to_nearest_postamat(model, *data):
    _, id_center_mass, postomat_place_id = data
    return model.center_mass_time_to_nearest_postamat[id_center_mass] >= distances_dict[(id_center_mass, postomat_place_id)] * model.has_postomat[postomat_place_id]

model.con_center_mass_time_to_nearest_postamat = pyo.Constraint( list(distances[['id_center_mass',	'object_id']].itertuples()) ,rule=con_center_mass_time_to_nearest_postamat)

def center_mass_has_postomat(model, center_mass_id):
    only_needed_dist = distances.loc[distances['id_center_mass'] == center_mass_id]
    out = 0
    for object_id in only_needed_dist['object_id']:
        out += model.has_postomat[object_id]
    return out


model.center_mass_has_postomat = pyo.Expression(population_points, rule=center_mass_has_postomat )



def con_center_mass_has_postomat(model, center_mass_id):
    return model.center_mass_time_to_nearest_postamat[center_mass_id] >= (1 - model.center_mass_has_postomat[center_mass_id]) * BIG_NUM 


model.con_center_mass_has_postomat = pyo.Constraint(population_points, rule=con_center_mass_has_postomat)



model.metro_time_to_nearest_postamat = pyo.Var(object_id_metro_list, within=pyo.NonNegativeReals)

def con_metro_time_to_nearest_postamat(model, *data):
    _, object_id_metro, postomat_place_id = data
    return model.metro_time_to_nearest_postamat[object_id_metro] >= distances_metro_dict[(object_id_metro, postomat_place_id)] * model.has_postomat[postomat_place_id]

model.con_metro_time_to_nearest_postamat = pyo.Constraint( list(distanses_metro[['object_id_metro',	'object_id']].itertuples()) ,rule=con_metro_time_to_nearest_postamat)


def metro_has_postomat(model, metro_id):
    only_needed_dist = distanses_metro.loc[distanses_metro['object_id_metro'] == metro_id]
    out = 0
    for object_id in only_needed_dist['object_id']:
        out += model.has_postomat[object_id]
    return out


model.metro_has_postomat = pyo.Expression(object_id_metro_list, rule=metro_has_postomat )

def con_metro_has_postomat(model, metro_id):
    return model.metro_time_to_nearest_postamat[metro_id] >= (1 - model.metro_has_postomat[metro_id]) * BIG_NUM 


model.con_metro_has_postomat = pyo.Constraint(object_id_metro_list, rule=con_metro_has_postomat)


model.needed_postamats = pyo.Constraint(expr=sum([model.has_postomat[p] for  p in postomat_places]) <= quantity_postamats_to_place)

sum_center_mass = sum(model.center_mass_time_to_nearest_postamat[p] * population_dict[p] for p in population_points) 
sum_metro = sum(model.metro_time_to_nearest_postamat[p] * population_dict[p] for p in object_id_metro_list)
# # Целевая
model.OBJ = pyo.Objective(expr=((1 - metro_weight) * sum_center_mass + (metro_weight) * sum_metro), sense=pyo.minimize)
# minimize

# , executable="/usr/local/Cellar/cbc/2.10.8/bin/cbc"
opt = SolverFactory('cbc')

for key in kwargs:
    opt.options[key] = kwargs[key]

results = opt.solve(model)


optimised_list = get_chosen_postomats(model)

optimised_list_no_fixed = list(set(optimised_list).difference(set(fixed_points)))